
# DATA SOURCES:

### NBA - Player Salary (1990-2017)
<a> https://www.kaggle.com/whitefero/nba-player-salary-19902017</a>
    
### Winningest Cities in Sports
<a> https://data.world/the-pudding/winningest-cities-in-sports/workspace/file?filename=titles.csv</a>   
   

In [1]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine

### Extract:

In [ ]:
#read in the csv files containing our data
csv_titles = "Resources/titles.csv"
titles_data_df = pd.read_csv(csv_titles)


csv_salaries = "Resources/salaries.csv"
salaries_data_df = pd.read_csv(csv_salaries)
titles_data_df

In [ ]:
salaries_data_df

### Transform

In [ ]:
#rename columns to remove spaces for easier querying
salaries_data_df = salaries_data_df.rename(columns={"Register Value": "Register_Value",
                                                    "Player Name": "Player_Name",
                                                    " Salary in $ ": "Salary_Dollars",
                                                    "Season Start": "Season_Start",
                                                    "Season End" :  "Season_End",
                                                    "Full Team Name" : "Full_Team_name"})
salaries_data_df

In [ ]:
#clean up for Boston Celtics team name so that team name listed the same in both data sets
salaries_data_df = salaries_data_df.replace({'Boston Celtic': 'Boston Celtics'})
salaries_data_df

In [ ]:
#remove all data from "winningest sports" data set not relating to profession basketball
titles_data_pro = titles_data_df.loc[titles_data_df["level"]=="pro"]
titles_data_sport = titles_data_pro.loc[titles_data_pro["sport"] == "NBA"]
titles_data_sport.dropna()

### Load

In [2]:
#Create connection to postgre
connection_string = "postgres:postgres@localhost:5432/NBA_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# add titles table
titles_data_sport.to_sql(name='titles', con=engine, if_exists='append', index=False)

In [ ]:
#add salaries table
salaries_data_df.to_sql(name='salaries', con=engine, if_exists='append', index=False)

In [3]:
#confirm titles table
pd.read_sql_query('select * from titles', con=engine)

,index,year,level,sport,winner,winner_metro,runner_up,runner_up_metro,final_four3,final_four3_metro,final_four4,final_four4_metro
0,243,1947,pro,NBA,Philadelphia Warriors,"Greater Philadelphia, PA",Chicago Stags,"Chicago, IL",New York Knicks,New York Metro Area,Washington Capitols,"Washington, DC Metro Area"
1,251,1948,pro,NBA,Baltimore Bullets,"Baltimore, MD",Philadelphia Warriors,"Greater Philadelphia, PA",Chicago Stags,"Chicago, IL",St. Louis Bombers,ERROR
2,259,1949,pro,NBA,Minneapolis Lakers,"Twin Cities, MN",Washington Capitols,"Washington, DC Metro Area",Rochester Royals,"Rochester, NY",New York Knicks,New York Metro Area
3,268,1950,pro,NBA,Minneapolis Lakers,"Twin Cities, MN",Syracuse Nationals,"Syracuse, NY",New York Knicks,New York Metro Area,Fort Wayne Pistons,"Fort Wayne, IN"
4,275,1951,pro,NBA,Rochester Royals,"Rochester, NY",New York Knicks,New York Metro Area,Minneapolis Lakers,"Twin Cities, MN",Syracuse Nationals,"Syracuse, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...
67,949,2014,pro,NBA,San Antonio Spurs,"San Antonio, TX",Miami Heat,"Greater Miami Area, FL",Indiana Pacers,"Indianapolis Metro Area, IN",Oklahoma City Thunder,"Oklahoma City, OK"
68,962,2015,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",Atlanta Hawks,"Atlanta, GA",Houston Rockets,"Houston, TX"
69,976,2016,pro,NBA,Cleveland Cavaliers,"Cleveland, OH",Golden State Warriors,"San Francisco Bay Area, CA",Toronto Raptors,"Toronto, ON",Oklahoma City Thunder,"Oklahoma City, OK"
70,989,2017,pro,NBA,Golden State Warriors,"San Francisco Bay Area, CA",Cleveland Cavaliers,"Cleveland, OH",San Antonio Spurs,"San Antonio, TX",Boston Celtics,"Greater Boston, MA"


In [4]:
#confirm salaries table
pd.read_sql_query('select * from salaries', con=engine)


,Register_Value,Player_Name,Salary_Dollars,Season_Start,Season_End,Team,Full_Team_name
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,354,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,727,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,1103,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,1497,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns
...,...,...,...,...,...,...,...
11832,6283,Zydrunas Ilgauskas,8740000,2005,2006,CLE,Cleveland Caveliers
11833,6658,Zydrunas Ilgauskas,9442697,2006,2007,CLE,Cleveland Caveliers
11834,7063,Zydrunas Ilgauskas,10142156,2007,2008,CLE,Cleveland Caveliers
11835,7485,Zydrunas Ilgauskas,10841615,2008,2009,CLE,Cleveland Caveliers


In [8]:
#join salaires and titles table to allow for queries about the players on the "winningest NBA team"
final_df = pd.read_sql_query('select * From salaries Left Join titles\
                            On salaries."Full_Team_name" = titles.winner\
                            Where salaries."Season_Start" = titles.year;', con=engine)
final_df

,Register_Value,Player_Name,Salary_Dollars,Season_Start,Season_End,Team,Full_Team_name,index,year,level,sport,winner,winner_metro,runner_up,runner_up_metro,final_four3,final_four3_metro,final_four4,final_four4_metro
0,1104,A.J. English,150000,1993,1994,CHI,Chicago Bulls,677,1993,pro,NBA,Chicago Bulls,"Chicago, IL",Phoenix Suns,"Metro Phoenix, AZ",New York Knicks,New York Metro Area,Seattle SuperSonics,"Seattle,WA"
1,7490,Adam Morrison,5257229,2009,2010,LAL,Los Angeles Lakers,885,2009,pro,NBA,Los Angeles Lakers,"Greater Los Angeles, CA",Orlando Magic,"Greater Orlando, FL",Cleveland Cavaliers,"Cleveland, OH",Denver Nuggets,"Denver, CO"
2,1503,Adrian Caldwell,150000,1994,1995,HOU,Houston Rockets,690,1994,pro,NBA,Houston Rockets,"Houston, TX",New York Knicks,New York Metro Area,Indiana Pacers,"Indianapolis Metro Area, IN",Utah Jazz,"Salt Lake City, UT"
3,5012,Alex Garcia,366931,2003,2004,SAS,San Antonio Spurs,806,2003,pro,NBA,San Antonio Spurs,"San Antonio, TX",New Jersey Nets,New York Metro Area,Dallas Mavericks,"Dallas-Fort Worth, TX",Detroit Pistons,"Metro Detroit, MI"
4,5432,Amal McCaskill,932546,2004,2005,DET,Detroit Pistons,819,2004,pro,NBA,Detroit Pistons,"Metro Detroit, MI",Los Angeles Lakers,"Greater Los Angeles, CA",Indiana Pacers,"Indianapolis Metro Area, IN",Minnesota Timberwolves,"Twin Cities, MN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1493,Will Perdue,1200000,1993,1994,CHI,Chicago Bulls,677,1993,pro,NBA,Chicago Bulls,"Chicago, IL",Phoenix Suns,"Metro Phoenix, AZ",New York Knicks,New York Metro Area,Seattle SuperSonics,"Seattle,WA"
376,348,William Bedford,850000,1990,1991,DET,Detroit Pistons,640,1990,pro,NBA,Detroit Pistons,"Metro Detroit, MI",Portland Trail Blazers,"Portland Metro Area, OR",Chicago Bulls,"Chicago, IL",Phoenix Suns,"Metro Phoenix, AZ"
377,8798,Yi Jianlian,827197,2011,2012,DAL,Dallas Mavericks,911,2011,pro,NBA,Dallas Mavericks,"Dallas-Fort Worth, TX",Miami Heat,"Greater Miami Area, FL",Chicago Bulls,"Chicago, IL",Oklahoma City Thunder,"Oklahoma City, OK"
378,1831,Zan Tabak,500000,1994,1995,HOU,Houston Rockets,690,1994,pro,NBA,Houston Rockets,"Houston, TX",New York Knicks,New York Metro Area,Indiana Pacers,"Indianapolis Metro Area, IN",Utah Jazz,"Salt Lake City, UT"


In [ ]:
final_df.count()

In [11]:
#Query the total salaries of the teams who've won for each year from 1990 - 2017
team_year = pd.read_sql_query('select titles.year, salaries."Full_Team_name",\
                            SUM (salaries."Salary_Dollars") AS Total_Team_Salary from salaries \
                            Left JOIN titles On salaries."Full_Team_name" = titles.winner\
                            Where salaries."Season_Start" = titles.year\
                            Group by year,"Full_Team_name"\
                            Order By year DESC,"Full_Team_name";', con=engine)
team_year

,year,Full_Team_name,total_team_salary
0,2017,Golden State Warriors,137494845.0
1,2015,Golden State Warriors,93707197.0
2,2014,San Antonio Spurs,69614583.0
3,2013,Miami Heat,82218984.0
4,2012,Miami Heat,81558311.0
5,2011,Dallas Mavericks,74463409.0
6,2010,Los Angeles Lakers,91428140.0
7,2009,Los Angeles Lakers,91378064.0
8,2008,Boston Celtics,79188973.0
9,2007,San Antonio Spurs,70048523.0


In [ ]:
#reformat data set to  avoid scientific notation
team_year = pd.options.display.float_format = '{:,.2f}'.format
team_year